# Gastebois Jacques
## Projet MTT : Distillation de Dataset par Correspondance de Trajectoires

Ce notebook constitue la partie principale de mon travail sur la reproduction du papier MTT. 
L'objectif est de condenser un dataset en un petit nombre d'images synthetiques tout en conservant une bonne performance de test.

**Note :** Ce notebook est une version epuree prevue pour tourner avec les buffers experts deja generes. 

---

### 1. initialisation et imports de base

on commence par charger tout le bousin necessaire. j'ai garde kornia pour les augmentations parce que c'est ce qui marche le mieux pour le mps sur mon m2. (enfin j'espere :))

In [ ]:
import os
import torch
import torch.nn as nn
import numpy as  np
from tqdm import tqdm
import copy
import  random

# imports du projet original
from utils import get_dataset, get_network, get_eval_pool, evaluate_synset, get_time, DiffAugment, ParamDiffAug
from reparam_module import  ReparamModule

def setup_device():
    # petite verif pour savoir ou on en est
    if torch.cuda.is_available():
        dev = 'cuda'
    elif   torch.backends.mps.is_available():
        dev = 'mps'
    else :
        dev = 'cpu'
    print(f"on tourne sur {dev} :)")
    return  dev
    
DEVICE = setup_device()

### 2. reglages et hyper-parametres

j'ai mis les reglages du papier par defaut mais pour les tests j'ai tendance a baisser le nombre d'iterations. les noms des variables sont pas forcement tres rigides mais on s'y retrouve. 

In [ ]:
class config:
    dataset = 'CIFAR10' 
    model =   'ConvNet'
    ipc = 1 # une image par classe c'est deja pas mal
    iter = 5000
    
    # reglages de l'optimiseur
    lr_img = 1000.0
    lr_net_param =  1e-05
    initial_lr = 0.01
    
    # chemins pour les donnees
    path_data = './data'
    # c'est ici qu'on met les buffers herberges sur hf
    path_buffer =  './buffers_c10_full'
    
    # parametres mtt
    expert_epochs = 3 
    syn_steps = 20
    max_start = 20
    
    zca = True # important pour cifar10 sinon ca converge pas top

args = config()
args.device = DEVICE
args.batch_real = 256
args.dsa_param = ParamDiffAug() # pour les augmentations

### 3. preparation des donnees

c'est la partie un peu penible ou on charge tout en memoire. faut etre patient :)

In [ ]:
print("chargement du dataset en cours...")

chan, size, classes, names, m, s, train_set, test_set, t_loader, loader_dict, c_map, c_map_inv = get_dataset(args.dataset, args.path_data, args.batch_real, '',  args=args)

# initialisation des images synthetiques
labels_syn = torch.tensor([np.ones(args.ipc,dtype=np.int_)*i for i in range(classes)], dtype=torch.long, requires_grad=False, device=DEVICE).view(-1)

# on part de quelque chose de realiste si possible
images_syn = torch.randn(size=(classes * args.ipc, chan, size[0],  size[1]), dtype=torch.float, device=DEVICE, requires_grad=True)

print(f"pret pour {classes} classes avec {args.ipc} images chaque")

# reglages des optimiseurs pour les images et le lr
opt_img = torch.optim.SGD([images_syn], lr=args.lr_img,  momentum=0.5)
lr_syn = torch.tensor(args.initial_lr).to(DEVICE).requires_grad_(True)
opt_lr = torch.optim.SGD([lr_syn], lr=args.lr_net_param, momentum=0.5)

print("optimiseurs ok :)")

### 4. boucle de distillation (mtt)

le gros du morceau. on essaye de copier ce que l'expert a fait pendant son entrainement.
si ca deconne au milieu c'est surement les buffers qui sont pas au bon endroit. 

In [ ]:
# chargement des trajectoires expertes
buffer_dir = os.path.join(args.path_buffer, args.dataset, "ConvNet")
print(f"recherche des experts dans {buffer_dir}")

# on charge le premier fichier pour voir
try:
    experts = torch.load(os.path.join(buffer_dir, "replay_buffer_0.pt"))
    print(f"on a {len(experts)} trajectoires chargees :)")
except:
    print("oups pas pu charger les buffers. verifie le chemin ou si t'as bien telecharge depuis hf")
    experts = [] 

def distillation_loop( iters):
    for it in  range(iters+1):
        
        # 1. on pioche un expert au pif
        trajectoire = random.choice(experts)
        
        # 2. on choisit un moment de sa vie (epoch)
        start_e = np.random.randint(0,  args.max_start)
        starting_params = trajectoire[start_e]
        target_params   = trajectoire[start_e + args.expert_epochs]
        
        # on met ca sous forme de vecteur plat
        target_p = torch.cat([p.data.to(DEVICE).reshape(-1) for p in target_params], 0)
        
        # 3. on cree notre eleve (student)
        eleve = get_network(args.model, chan, classes, size).to(DEVICE)
        eleve = ReparamModule(eleve)
        eleve.train()

        # on lui donne les parametres de depart de l'expert
        p_eleve = [torch.cat([p.data.to(DEVICE).reshape(-1) for p in starting_params], 0).requires_grad_(True)]
        
        # 4. on fait faire des pas a l'eleve sur nos images bidon
        for  step in range(args.syn_steps):
             # petites augmentations au passage
             x = DiffAugment(images_syn, 'color_crop_cutout_flip_scale_rotate', param=args.dsa_param)
             
             out = eleve(x, flat_param=p_eleve[-1])
             loss = nn.CrossEntropyLoss()(out, labels_syn)
             
             # calcul du gradient pour le student
             grad = torch.autograd.grad(loss, p_eleve[-1], create_graph=True)[0]
             p_eleve.append(p_eleve[-1] -  lr_syn * grad)
             
        # 5. on regarde si on est loin de l'expert
        perte_mtt = nn.functional.mse_loss(p_eleve[-1], target_p, reduction="sum")
        
        # mise a jour de nos images synthetiques
        opt_img.zero_grad()
        opt_lr.zero_grad()
        
        perte_mtt.backward()
        
        opt_img.step()
        opt_lr.step()
        
        if  it % 10 == 0:
            print(f"iteration {it} - perte : {perte_mtt.item():.4f} :)")

# a lancer quand t'es pret
# distillation_loop(args.iter)